In [50]:
import torch
import seaborn
import pandas as pd
from sklearn import metrics
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import sentencepiece

In [51]:
#Chargement du jeu de données
df = pd.read_csv("emotion_dataworld.csv")

In [52]:
df = df[0:10000]

In [53]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [54]:
df = df.rename(columns={"content": "review"})

In [55]:
df.drop( df[ (df['sentiment'] == 'neutral')].index, inplace=True)

In [56]:
df = df.reset_index()

In [57]:
df['sentiment'][df['sentiment']== 'worry'] = 0
df['sentiment'][df['sentiment']== 'happiness'] = 1
df['sentiment'][df['sentiment']== 'sadness'] = 0
df['sentiment'][df['sentiment']== 'love'] = 1
df['sentiment'][df['sentiment']== 'surprise'] = 1
df['sentiment'][df['sentiment']== 'fun'] = 1
df['sentiment'][df['sentiment']== 'relief'] = 1
df['sentiment'][df['sentiment']== 'hate'] = 0
df['sentiment'][df['sentiment']== 'empty'] = 0
df['sentiment'][df['sentiment']== 'enthusiasm'] = 1
df['sentiment'][df['sentiment']== 'boredom'] = 0
df['sentiment'][df['sentiment']== 'anger'] = 0

C:\Users\APPREN~1\AppData\Local\Temp/ipykernel_22032/3945871787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'][df['sentiment']== 'worry'] = 0
C:\Users\APPREN~1\AppData\Local\Temp/ipykernel_22032/3945871787.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'][df['sentiment']== 'happiness'] = 1
C:\Users\APPREN~1\AppData\Local\Temp/ipykernel_22032/3945871787.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [58]:
df['sentiment'] = df['sentiment'].astype(int)

In [59]:
dataset = df.drop(['index','tweet_id','author'], axis = 1)

In [60]:
dataset

,sentiment,review
0,0,@tiffanylue i know i was listenin to bad habi...
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
3,1,wants to hang out with friends SOON!
4,0,Re-pinging @ghostridah14: why didn't you go to...
...,...,...
8138,0,http://twitpic.com/674p1 - @JTHawthorne this i...
8139,0,is sadly sitting at home when she could be goi...
8140,0,@ArlenesUniverse I have tried. He doesn't want...
8141,0,#twitterfails fucking hard right now...this is...


In [61]:

 
reviews = dataset['review'].values.tolist()
sentiments = dataset['sentiment'].values.tolist()
 
# On charge l'objet "tokenizer"de camemBERT qui va servir a encoder
# 'camebert-base' est la version de camembert qu'on choisit d'utiliser
# 'do_lower_case' à True pour qu'on passe tout en miniscule
TOKENIZER = BertTokenizer.from_pretrained(
    'bert-base-cased',
    do_lower_case=True)
 
# La fonction batch_encode_plus encode un batch de donnees
encoded_batch = TOKENIZER.batch_encode_plus(reviews,
                                            add_special_tokens=True,
                                            max_length=30,
                                            padding=True,
                                            truncation=True,
                                            return_attention_mask = True,
                                            return_tensors = 'pt')
 
# On transforme la liste des sentiments en tenseur
sentiments = torch.tensor(sentiments)
 
# On calcule l'indice qui va delimiter nos datasets d'entrainement et de validation
# On utilise 80% du jeu de donnée pour l'entrainement et les 20% restant pour la validation
split_border = int(len(sentiments)*0.8)
 
 
train_dataset = TensorDataset(
    encoded_batch['input_ids'][:split_border],
    encoded_batch['attention_mask'][:split_border],
    sentiments[:split_border])
validation_dataset = TensorDataset(
    encoded_batch['input_ids'][split_border:],
    encoded_batch['attention_mask'][split_border:],
    sentiments[split_border:])
 
 
batch_size = 128
 
# On cree les DataLoaders d'entrainement et de validation
# Le dataloader est juste un objet iterable
# On le configure pour iterer le jeu d'entrainement de façon aleatoire et creer les batchs.
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size)
 
validation_dataloader = DataLoader(
            validation_dataset,
            sampler = SequentialSampler(validation_dataset),
            batch_size = batch_size)

In [62]:
# On la version pre-entrainee de camemBERT 'base'
model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels = 2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [63]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # Learning Rate
                  eps = 1e-8) # Epsilon
epochs = 4

In [64]:
# On va stocker nos tensors sur mon cpu : je n'ai pas mieux
device = torch.device("cpu")
 
# Pour enregistrer les stats a chaque epoque
training_stats = []
 
# Boucle d'entrainement
for epoch in range(0, epochs):
     
    print("")
    print(f'########## Epoch {epoch+1} / {epochs} ##########')
    print('Training...')
 
 
    # On initialise la loss pour cette epoque
    total_train_loss = 0
 
    # On met le modele en mode 'training'
    # Dans ce mode certaines couches du modele agissent differement
    model.train()
 
    # Pour chaque batch
    for step, batch in enumerate(train_dataloader):
 
        # On fait un print chaque 40 batchs
        if step % 40 == 0 and not step == 0:
            print(f'  Batch {step}  of {len(train_dataloader)}.')
         
        # On recupere les donnees du batch
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        sentiment = batch[2].to(device)
 
        # On met le gradient a 0
        model.zero_grad()        
 
        # On passe la donnee au model et on recupere la loss et le logits (sortie avant fonction d'activation)
        loss, logits = model(input_id, 
                             token_type_ids=None, 
                             attention_mask=attention_mask, 
                             labels=sentiment)
 
        # On incremente la loss totale
        # .item() donne la valeur numerique de la loss
        total_train_loss += loss.item()
 
        # Backpropagtion
        loss.backward()
 
        # On actualise les parametrer grace a l'optimizer
        optimizer.step()
 
    # On calcule la  loss moyenne sur toute l'epoque
    avg_train_loss = total_train_loss / len(train_dataloader)   
 
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))  
     
    # Enregistrement des stats de l'epoque
    training_stats.append(
        {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
        }
    )
 
print("Model saved!")
torch.save(model.state_dict(), "./sentiments.pt")


########## Epoch 1 / 4 ##########
Training...
  Batch 40  of 51.

  Average training loss: 0.57

########## Epoch 2 / 4 ##########
Training...
  Batch 40  of 51.

  Average training loss: 0.52

########## Epoch 3 / 4 ##########
Training...
  Batch 40  of 51.

  Average training loss: 0.46

########## Epoch 4 / 4 ##########
Training...
  Batch 40  of 51.

  Average training loss: 0.36
Model saved!
